# INFO370 Problem Set 6: Linear Regression

In [207]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as smf

## 1 Data description (15pt)

#### 1. Load the data airbnb-seattle-listings-train.csv. Broadly describe the variables you see, their encoding, and discuss if these may be valuable in determining the price. For instance, you may want to thell that house_rules is text, and you may want to check if smoking allowed/not allowed is related to the price.

In [287]:
listings = pd.read_csv('airbnb-seattle-listings-train.csv', sep='\t')

After reading in the data, we learned that there are 106 variables for each airbnb. Some of these variables are only useful for backend such as 'id' and 'listing_url' while other variables such as 'description,' 'review_scores,' 'weekly_price,' and 'monthly_price' are useful for determining price. One example of a useful variable is 'review_scores_cleanliness' which is a float that tells the user the rating of how clean the airbnb is. The cleanliness score allows the seller to market their airbnb at a higher price.

#### 2. Consider how will you handle missing data. For instance, 95% of the 'square feet' observations are missing, 17% of 'security deposit' observations are missing. You lose too many observations if you just ignore those.

In [288]:
listings = listings.drop("square_feet", axis=1)
listings = listings.drop("thumbnail_url", axis=1)
listings = listings.drop("medium_url", axis=1)
listings = listings.drop("xl_picture_url", axis=1)
listings = listings.drop("host_acceptance_rate", axis=1)
listings = listings.drop("license", axis=1)
listings = listings.drop("host_about", axis=1)
listings = listings.drop("weekly_price", axis=1)
listings = listings.drop("monthly_price", axis=1)

Our group decided to delete any variables that are missing more than 20% of data and do not seem useful to the project. There are 15 out of 106 variables that are missing this much data, and we decided that 9 of these are unnecessary to the project. These variables included LIST A FEW THAT WE DELETED AND WHY.

#### 3. Consider which variables you are going to use below. For all of these, create a summary table that contains relevant summary information. In particular pay attention to the missing values. Note that missings may not just be coded as such, they may also be empty strings and values like 'N/A.' You may return to this point repeatedly as you develop your model.

The variables that we chose were beds, security deposit, zipcode, and score rating.

In [315]:
listings.security_deposit[:10]
#isinstance(listings.security_deposit[1], str)

0    $500.00
1    $120.00
2    $100.00
3      $0.00
4    $300.00
5    $300.00
6    $200.00
7    $300.00
8    $350.00
9        NaN
Name: security_deposit, dtype: object

In [289]:
listings['security'] = listings.security_deposit

In [311]:
for i in range(0, len(listings.security_deposit)):
    var = listings.security_deposit[i]
    
    if (isinstance(var, str)):
        var = listings.security_deposit[i][1:]
        var = var.replace(',', '')
        var = float(var)
    
    
    listings.security[i] = var

C:\Users\kyler\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [316]:
# REPLACE NA WITH SECURITY DEPOSIT MEANS
# THIS DOESNT WORK YET
mean_deposit = listings.security.mean
mean_deposit
listings.security.fillna(value = mean_deposit)
listings.security

0        500
1        120
2        100
3          0
4        300
5        300
6        200
7        300
8        350
9        NaN
10         0
11       500
12       500
13         0
14         0
15       300
16       500
17       500
18      1500
19       350
20         0
21       100
22         0
23       100
24       NaN
25       100
26        95
27       250
28       NaN
29       100
        ... 
7510     400
7511     NaN
7512     300
7513       0
7514     NaN
7515     500
7516       0
7517     200
7518     200
7519     NaN
7520     300
7521       0
7522     500
7523     250
7524     NaN
7525     NaN
7526     NaN
7527     150
7528       0
7529     400
7530     100
7531     NaN
7532     NaN
7533     NaN
7534       0
7535     NaN
7536     NaN
7537     NaN
7538       0
7539       0
Name: security, Length: 7540, dtype: object

In [212]:
listings.security = listings.security.astype('float')

In [213]:
listings.security.describe()

count    7540.00000
mean      213.62321
std       367.10436
min        -1.00000
25%         0.00000
50%       100.00000
75%       300.00000
max      5000.00000
Name: security, dtype: float64

In [214]:
listings.security.mode()

0    0.0
dtype: float64

In [267]:
listings.beds.fillna(value = 1.0)

0       4.0
1       1.0
2       1.0
3       1.0
4       3.0
5       2.0
6       4.0
7       5.0
8       3.0
9       1.0
10      3.0
11      1.0
12      1.0
13      1.0
14      1.0
15      3.0
16      2.0
17      2.0
18      3.0
19      2.0
20      1.0
21      1.0
22      1.0
23      1.0
24      3.0
25      1.0
26      2.0
27      1.0
28      5.0
29      1.0
       ... 
7510    4.0
7511    3.0
7512    2.0
7513    1.0
7514    1.0
7515    1.0
7516    2.0
7517    1.0
7518    1.0
7519    1.0
7520    1.0
7521    3.0
7522    4.0
7523    3.0
7524    1.0
7525    1.0
7526    1.0
7527    1.0
7528    3.0
7529    3.0
7530    1.0
7531    1.0
7532    1.0
7533    1.0
7534    1.0
7535    2.0
7536    4.0
7537    0.0
7538    1.0
7539    3.0
Name: beds, Length: 7540, dtype: float64

In [215]:
# Amount of beds
listings.beds.describe()

count    7537.000000
mean        1.908850
std         1.561733
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        49.000000
Name: beds, dtype: float64

In [216]:
listings.beds.mode()

0    1.0
dtype: float64

In [220]:
listings['zipcode'].mode()
# Most rentals are in zipcode 98122 (mode)

0    98122
dtype: object

In [219]:
listings['review_scores_rating'].mode()

0    100.0
dtype: float64

There was not a lot cleaning we needed to do to the data because we chose variables that were both representative of things we consider important for price (like location, area, rating) and also fairly clean.
Most of the cleaning involved resolving NaN values to something usable and converting strings to numeric values (e.g  for security deposits, filling NaN with $-1, stripping dollar sign, converting to float value)

## 2 Model (60pt)

#### 1. Either split your data into training and validation sets, or just use cross validation below

In [268]:
import sklearn.linear_model as slm
from sklearn.model_selection import train_test_split

In [269]:
for i in range(0, len(listings.price)):
    var = listings.price[i][1:]
    listings.price[i] = var.replace(',', '')

IndexError: invalid index to scalar variable.

In [262]:
listings.price = listings.price.astype('float')

False    7540
Name: price, dtype: int64

In [245]:
# beds, security deposit, and review score rating
X = np.stack((listings.beds.values, listings.security.values, listings.review_scores_rating.values), axis = 1)
y = listings.price

In [247]:
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size = 0.2)

In [248]:
Xtr.shape

(6032, 3)

#### 2. Develop the models. Report all the variables and how do you clean/encode those. While the exact details are visible in the code, explain the broad choices in text.

In [263]:
m = slm.LinearRegression().fit(Xtr[:,:1], ytr)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

#### 3. Report the final number of observations, the estimated coefficient values, adjusted R2, and RMSE on validataion data for three models:

(a) a simple one that only contains a few most important variables/best predictors. What do you think are 2-3 best predictors in the data?

In [ ]:
Xtr = 

(b) the full model: everything you consider useful.

(c) something in between.

#### 4. Interpret the coefficients of the reported models. Again, only interpret the most interesting/important ones, not all of those! Do the coefficient values differ between the models? Can you explain why?

In [ ]:
m = smf.ols(formula = 'beds ~ price', data = listings).fit()
m.summary()

In [7]:
(((listings.isnull().sum() / (listings.isnull().sum() + listings.notnull().sum())) * 100) > 20).value_counts()

False    91
True      6
dtype: int64